## Importing multi-sample VCF (Illumina-style) into Hail

This tutorial is for Hail v0.2 https://hail.is/docs/devel/index.html.

In [1]:
import hail as hl
import hail.expr.aggregators as agg
hl.init()

Running on Apache Spark version 2.2.0
SparkUI available at http://10.46.229.155:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version devel-2610434c191b
NOTE: This is a beta version. Interfaces may change
  during the beta period. We recommend pulling
  the latest changes weekly.


Import block-compressed VCF, split multi-allelic sites and perform variant + sample QC.

In [2]:
vcf='polaris.chr20.hail.vcf.bgz'
vt = hl.import_vcf(vcf)
vt = hl.split_multi_hts(vt)
vt = hl.variant_qc(vt)
vt = hl.sample_qc(vt).cache()

2018-09-21 11:22:32 Hail: INFO: Coerced almost-sorted dataset
2018-09-21 11:23:12 Hail: INFO: Coerced almost-sorted dataset


In [10]:
cnt = vt.count()
print("Have {} samples and {} variants".format(cnt[0],cnt[1]))
vt.rows().select().show(5)

Have 984253 samples and 148 variants
+---------------+------------+
| locus         | alleles    |
+---------------+------------+
| locus<GRCh37> | array<str> |
+---------------+------------+
| 20:60006      | ["A","C"]  |
| 20:60008      | ["A","C"]  |
| 20:60020      | ["A","C"]  |
| 20:60024      | ["A","G"]  |
| 20:60053      | ["G","T"]  |
+---------------+------------+
showing top 5 rows



Variant and sample QC create additional row and column fields, respectively. Examples are call rate (% calls for a variant across samples), ratio of transitions vs transversion etc.

In [4]:
from pprint import pprint
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import Span
output_notebook()

Loading BokehJS ...

Plot some mrtrics such as call rate (fraction of non-missing calls), Transition/Transversions and Het/Hom ratio.

In [5]:
call_rate_mean = vt.aggregate_rows(agg.mean(vt.variant_qc.call_rate))
print('Call rate mean=%.3f' % call_rate_mean)
p = hl.plot.histogram(vt.variant_qc.call_rate, range=(0,1.0), bins=30, title='Call rate Histogram', legend='Call Rate')
show(p)

Call rate mean=0.989


Ti/Tv (sometimes called Ts/Tv): the ratio of transitions vs. transversions in SNPs. We expect this ratio to be close to 2.1 for whole-genome sequencing

In [11]:
r_ti_tv_mean = vt.aggregate_cols(agg.mean(vt.sample_qc.r_ti_tv))
print("Ti/Tv mean=%.3f" % r_ti_tv_mean)
p = hl.plot.histogram(vt.sample_qc.r_ti_tv, range=(1,3), bins=30, title='Ti/Tv Histogram', legend='Ti/Tv')
show(p)

Ti/Tv mean=2.054


Plot a histogram of the ratio of heterozygous vs homozygous variants across samples. We expect this ratio to be approximately 2:1.

In [7]:
r_het_hom_var_mean = vt.aggregate_cols(agg.mean(vt.sample_qc.r_het_hom_var))
print('Het/hom var mean=%.2f' % r_het_hom_var_mean)
p = hl.plot.histogram(vt.sample_qc.r_het_hom_var, range=(0,5), bins=30, title='Histogram of Het/Hom ratios', legend='Ti/Tv')
show(p)

Het/hom var mean=2.31


Finally, we write all variants (with QC annotations) to disk. This is now Hail's internal format which is faster to read from for future analyses.

In [8]:
outfile='polaris.chr20.diplofy.mt'
vt.write(outfile,overwrite=True) 

2018-09-21 11:42:56 Hail: INFO: wrote 984253 items in 20 partitions to polaris.chr20.diplofy.mt
